# Классификатор на основе модели от Google

Модель:
*  https://github.com/mmihaltz/word2vec-GoogleNews-vectors

In [0]:
from gensim.models import KeyedVectors
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import collections
import nltk
import numpy as np
import codecs
import pandas as pd
import gensim
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten, Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Activation
from keras.models import Model
import sklearn
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import save_model, load_model
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import RegexpTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def tokenize(text):
  regex_tokenizer = RegexpTokenizer('[a-zA-Z]+')
  words = regex_tokenizer.tokenize(text.lower())
  stop_words = set(stopwords.words("english"))
  without_stop_words = [w for w in words if w not in stop_words]
  return without_stop_words

In [0]:
def transform_from_categorical(prediction, intents):
  answers = []

  for i in range(prediction.shape[0]):
    answers.append(intents[prediction[i]])

  return answers

def transform_to_categorical(intents_for_each, unique_intents):
  intents_to_digit = []

  for intent in intents_for_each:
    intents_to_digit.append(unique_intents.index(intent))

  return to_categorical(intents_to_digit, len(unique_intents))

In [11]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Загрузка данных и модели от Google

In [5]:
file_csv = pd.read_csv('drive/My Drive/ForGensim/train.csv')
word2vec_model = KeyedVectors.load_word2vec_format(
    "drive/My Drive/ForGensim//GoogleNews-vectors-negative300.bin.gz",
    binary=True)

EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
NUM_EPOCHS = 10

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
counter = collections.Counter()
maxlen = 0

Просмотрели некоторые слова в словаре

In [173]:
from itertools import islice
list(islice(word2vec_model.vocab, 15010, 15020))

['John_F._Kennedy',
 'ideals',
 'insane',
 'Dow_Jones_Industrial_Average',
 'Guillen',
 'Established',
 'lip',
 'SS',
 'Drop',
 'prominence']

Работа с набором данных (удаление стоп-слов, получение частотного словаря, максимальной длины)

In [0]:
all_texts = file_csv['text']
all_intents = file_csv['intents']

In [0]:
sentences = np.array(all_texts.apply(lambda x : tokenize(x)))

In [0]:
for words in sentences:
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1

In [0]:
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    word2index[word[0]] = wid + 1
vocab_sz = len(word2index) + 1
index2word = {v: k for k, v in word2index.items()}

Подготовка данных для обучения на основе модели от Google

In [0]:
xs, ys = [], []
for i in range(0, len(sentences)):
  wids = [word2index[word] for word in sentences[i]]
  xs.append(wids)

In [0]:
ys = transform_to_categorical(all_intents, all_intents.unique().tolist())

In [0]:
X = pad_sequences(xs, maxlen=maxlen)
Y = ys

In [78]:
Xtrain, Xtest, Ytrain, Ytest = \
    train_test_split(X, Y, test_size=0.3, random_state=42)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)

(7961, 22) (3413, 22) (7961, 5) (3413, 5)


Создание keras модели

In [0]:
embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
    try:
        embedding_weights[index, :] = word2vec_model[word]
    except KeyError:
        pass

In [0]:
COUNT_CLASSES = all_intents.unique().shape[0]

In [0]:
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen,
                    weights=[embedding_weights],
                    trainable=True))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS,
                 activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(COUNT_CLASSES, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

In [90]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 22, 300)           1500300   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 22, 300)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 20, 256)           230656    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 1285      
Total params: 1,732,241
Trainable params: 1,732,241
Non-trainable params: 0
_________________________________________________________________


Обучение keras модели

In [91]:
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=[ModelCheckpoint('drive/My Drive/ForGensim/google_keras_model.h5', save_best_only = True)],
                    validation_data=(Xtest, Ytest))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7961 samples, validate on 3413 samples
Epoch 1/10
7961/7961 [==============================] - 14s 2ms/step - loss: 0.2016 - acc: 0.9567 - val_loss: 0.0375 - val_acc: 0.9927
Epoch 2/10
7961/7961 [==============================] - 13s 2ms/step - loss: 0.0258 - acc: 0.9952 - val_loss: 0.0230 - val_acc: 0.9950
Epoch 3/10
7961/7961 [==============================] - 13s 2ms/step - loss: 0.0116 - acc: 0.9985 - val_loss: 0.0170 - val_acc: 0.9968
Epoch 4/10
7961/7961 [==============================] - 13s 2ms/step - loss: 0.0057 - acc: 0.9992 - val_loss: 0.0141 - val_acc: 0.9965
Epoch 5/10
7961/7961 [==============================] - 13s 2ms/step - loss: 0.0034 - acc: 0.9996 - val_loss: 0.0135 - val_acc: 0.9971
Epoch 6/10
7961/7961 [==============================] - 13s 2ms/step - loss: 0.0022 - acc: 0.9999 - val_loss: 0.0133 - val_acc: 0.9974
Epoch 7/10
7961/7961 [========================

In [92]:
score = model.evaluate(Xtest, Ytest, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

3413/3413 [==============================] - 1s 245us/step
Test score: 0.013, accuracy: 0.997


Модель обучилась с точностью ~99%

# Тестирование (проверка работы) классификатора на небольших данных

In [0]:
test = ['Add track to my Digster Future Hits', 'what a temperature today in kirov', 'add to playlist my song my melody', 
        'What is the hottest temperature on earth right now?', 'Find a movie schedule for 12 hours from now'] 

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [168]:
sequences_test = tokenizer.texts_to_sequences(test)
sequences_test

[[2, 21, 576, 617, 111],
 [572, 147],
 [2, 3, 18, 193],
 [572, 1039, 231],
 [13, 5, 17, 87]]

In [0]:
X_predict = pad_sequences(sequences_test, maxlen=maxlen)

In [0]:
prediction = model.predict_classes(X_predict)

In [171]:
answers = transform_from_categorical(prediction, all_intents.unique().tolist()) 
answers

['AddToPlaylist',
 'GetWeather',
 'AddToPlaylist',
 'GetWeather',
 'SearchScreeningEvent']